# Deep Networks (Draft)
> Part 5 of the mangoes_blog project

- branch: master
- toc: true 
- badges: false
- comments: false
- sticky_rank: 5
- author: Huon Fraser
- categories: [mangoes]

In [1]:
from copy import deepcopy
from sklearn.metrics import r2_score, mean_squared_error

import sys
sys.path.append('/notebooks/Mangoes/src/')
model_path  = '../models/'


## Cross-Validation

So far we have implemented a minimum-viable model and evaluation. We now wrap this into a cross-validation framework.

First we need to consider a key design choice. For our sklearn models, cross-validation led to building multiple versions of a model with slightly different parameters. For neural networks, with optimising being a highly stochastic gradient descent path, there is no gurantee that cross-validation folds are similar, or that folds resemble the final model trained on all the data. After running cross-validation, we let the user define how to build the final model; None, for building no final model to save time, "All", to train a model on the whole training set, or "Ensemble", to build an ensemble on the cross-validation folds. 

We define our ensemble implementation below. At the moment we just pass in each model. IN the future we may instead pass in a location of a savefile for each model, or a single model and a list of locations to get weights from.

In [ ]:
@dataclass
class EnsembleConfig(ModelConfig):
    models: list  = []
    voting : str = "Mean"

class Ensemble(EnsembleModel):
    def __init__(
        self,
        config: DictConfig,
        **kwargs
    ):
        super().__init__(config, **kwargs)

    def _build_network(self):
        self.models = self.hparams["continuous_dim"]
        
    def forward(self,x):
        x = x["continuous"]
        y_hats=torch.zeros(x.shape[1],len(self.models))
        for i,model in enumerate(models):
            y_hats[:,i]= self.model.forward(x)['logits']
        if voting == "Mean":
            y_hat = torch.mean(y_hats)
        return  {'logits':y_hat}

In [ ]:
from sklearn.model_selection import GroupKFold, KFold

def cross_validate(model,X,y,splitter=GroupKFold(),groups=None,plot=False,save_loc=None,final_model="Ensemble"): #Ensemble, "All", None
    #combine X and y
    Xy = deepcopy(X)
    Xy["target"]=y
    
    preds = None
    ys = None
    models = []
    for fold, (inds1,inds2) in enumerate(splitter.split(X,y,groups)):
        
        model.fit(X.iloc[inds1,:],y.iloc[inds1,:])
        pred = model.predict(X.iloc[inds2,:])

        if preds is None:
            preds = pred
            ys = y.iloc[inds2,:]
        else:
            preds = np.concatenate((preds,pred),axis=0)
            ys = np.concatenate((ys,y.iloc[inds2,:]),axis=0)
            
        if final_model == "Ensemble":
            models.append(deepcopy(model))
            

    r2 = r2_score(ys,preds)
    mse = mean_squared_error(ys,preds)

    if plot:
        ys = ys.flatten()
        preds = preds.flatten()

        m, b = np.polyfit(ys, preds, 1)
        fig, ax = plt.subplots()

        ls = np.linspace(min(ys),max(ys))
        ax.plot(ls,ls*m+b,color = "black", label = r"$\hat{y}$ = "+f"{m:.4f}y + {b:.4f}")
        ax.scatter(x=ys,y=preds,label = r"$R^2$" + f"={r2:.4f}")

        ax.set_xlabel('True Values')
        ax.set_ylabel('Predicted Values')
        ax.legend(bbox_to_anchor=(0.5,1))
        if not save_loc is None:
            fig.savefig(save_loc)
            
    if final_model == "Ensemble":
         #create new tabular model, passing in same configs but with an ensemble
            
        ens_config = EnsembleConfig(models=models)
            
        ensemble = TabularModel(
            data_config=model.data_config,
            optimizer_config=model.optimizer_config,
            trainer_config=model.trainer_config,
            model_config=ens_config,
            model_callable = Ensemble
)
        model = ensemble
    elif final_model == "All": #train final model on all data
        model = model.fit(train=Xy)
    else: #ignore training the final model, for computation saving purposes 
        model = None
    
    return model, mse

In [ ]:
def evaluate(model,train_X,train_y,test_X,test_y,plot=False,save_loc=None,log=True):
    test_y=test_y.values.flatten()
    model.fit(train_X,train_y)
    preds = model.predict(test_X)

    r2 = r2_score(test_y,preds)
    mse = mean_squared_error(test_y,preds)

    if log:
        print(f"Test set MSE: {mse:.4f}")

    if plot:
        preds=preds.flatten()

        m, b = np.polyfit(test_y, preds, 1)
        fig, ax = plt.subplots()

        ls = np.linspace(min(test_y),max(test_y))
        ax.plot(ls,ls*m+b,color = "black", label = r"$\hat{y}$ = "+f"{m:.4f}y + {b:.4f}")
        ax.scatter(x=test_y,y=preds,label = r"$R^2$" + f"={r2:.4f}")

        ax.set_xlabel('True Values')
        ax.set_ylabel('Predicted Values')
        ax.legend(bbox_to_anchor=(0.5,1))
        if not save_loc is None:
            fig.savefig(save_loc)
    return model, mse 